---------------------------------------------------------
1. IMPORTS Y CONFIGURACIÓN DE AURA DB 
---------------------------------------------------------

In [1]:
import pandas as pd
from neo4j import GraphDatabase
import os

URI = "neo4j+s://ce30717d.databases.neo4j.io"
AUTH = ("neo4j", "qqxxUQXCSnR9yp3X9YLDpgKbwaQOVtjXS83kY6tKY6s")

---------------------------------------------------------
2. LEEMOS LOS CSVS GENERADOS 
---------------------------------------------------------

In [2]:
users_df = pd.read_csv('../data/users.csv')
sessions_df = pd.read_csv('../data/sessions.csv')
events_df = pd.read_csv('../data/events.csv')

# Convertir DataFrames a lista de diccionarios (formato nativo para Neo4j)
# Convertimos timestamps a string ISO o se pueden pasar como parámetros datetime
users_data = users_df.to_dict('records')
sessions_data = sessions_df.to_dict('records')
events_data = events_df.to_dict('records')

---------------------------------------------------------
3. DEFINIR QUERIES
---------------------------------------------------------

In [3]:
# 1. Constraints (Se ejecutan una vez)
q_constraints = [
    "CREATE CONSTRAINT IF NOT EXISTS FOR (u:User) REQUIRE u.user_id IS UNIQUE",
    "CREATE CONSTRAINT IF NOT EXISTS FOR (v:Visitor) REQUIRE v.id IS UNIQUE",
    "CREATE CONSTRAINT IF NOT EXISTS FOR (s:Session) REQUIRE s.session_id IS UNIQUE",
    "CREATE CONSTRAINT IF NOT EXISTS FOR (e:Event) REQUIRE e.event_id IS UNIQUE"
]

# 2. Cargar Usuarios
q_users = """
UNWIND $rows AS row
MERGE (u:User {user_id: row.user_id})
SET u.first_name = row.first_name,
    u.last_name = row.last_name,
    u.email = row.email,
    u.city = row.city,
    u.registration_date = datetime(replace(row.registration_date, ' ', 'T'))
"""

# 3. Cargar Sesiones + User Stitching
q_sessions = """
UNWIND $rows AS row
MERGE (s:Session {session_id: row.session_id})
SET s.date = datetime(replace(row.session_date, ' ', 'T')),
    s.source = row.source,
    s.medium = row.medium,
    s.device = row.device

MERGE (v:Visitor {id: row.user_pseudo_id})
MERGE (v)-[:INICIO_SESION]->(s)

WITH row, s, v
WHERE row.user_id IS NOT NULL AND row.user_id <> "" AND toString(row.user_id) <> "nan"
MATCH (u:User {user_id: row.user_id})
MERGE (u)-[:REALIZO]->(s)
MERGE (v)-[:IDENTIFIED_AS]->(u)
"""

# 4. Cargar Eventos
q_events = """
UNWIND $rows AS row
MERGE (e:Event {event_id: row.event_id})
SET e.name = row.event_name,
    e.timestamp = datetime(replace(row.event_timestamp, ' ', 'T')),
    e.url = row.page_location,
    e.title = row.page_title

WITH row, e
MATCH (s:Session {session_id: row.session_id})
MERGE (s)-[:CONTIENE]->(e)
"""

---------------------------------------------------------
4. FUNCIÓN DE EJECUCIÓN
---------------------------------------------------------

In [4]:
def load_data():
    print("Conectando a AuraDB...")
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("Conexión exitosa.")
        
        with driver.session() as session:
            # --- PASO DE LIMPIEZA ---
            print("Limpieza previa de la base de datos...")
            session.run("MATCH (n) DETACH DELETE n")

            # 1. Constraints
            print("Creando índices...")
            for q in q_constraints:
                session.run(q)
            
            # 2. Users
            print(f"Cargando {len(users_data)} usuarios...")
            session.run(q_users, rows=users_data)
            
            # 3. Sessions (Importante: pandas a veces deja NaNs, limpiamos un poco antes si es necesario, 
            # pero la query ya maneja nulos con WHERE)
            print(f"Cargando {len(sessions_data)} sesiones...")
            # Pre-limpieza de NaN para user_id en Python para evitar errores de tipo
            cleaned_sessions = []
            for s in sessions_data:
                s_clean = s.copy()
                if pd.isna(s_clean['user_id']):
                    s_clean['user_id'] = None
                cleaned_sessions.append(s_clean)
                
            session.run(q_sessions, rows=cleaned_sessions)
            
            # 4. Events
            print(f"Cargando {len(events_data)} eventos...")
            session.run(q_events, rows=events_data)

            # 5. Cerramos la sesión
            session.close()

        # 6. Cerramos el driver
        driver.close()

    print("¡Carga completa!")    

if __name__ == "__main__":
    load_data()

Conectando a AuraDB...
Conexión exitosa.
Limpieza previa de la base de datos...
Creando índices...
Cargando 100 usuarios...
Cargando 500 sesiones...
Cargando 2188 eventos...
¡Carga completa!
